# Analysis


In [39]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#load csv into dataframe
loans_data = pd.read_csv('Data_Post_Transformation.csv')
loans_data.drop('Unnamed: 0', axis='columns', inplace=True)

log_convert_back = ['recoveries',
                    'total_rec_late_fee',
                    'delinq_2yrs',
                    'inq_last_6mths',
                    'annual_inc',
                    'out_prncp',
                    'out_prncp_inv',
                    'last_payment_amount',
                    'collections_12_mths_ex_med',
                    'collection_recovery_fee']

#restore log_converted columns back to pre-transformation values

for col in log_convert_back:
    loans_data[col] = loans_data[col].map(lambda x: np.exp(x) if x > 0 else 0)

pd.set_option('display.max_columns', None)

display(loans_data.head())

,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,employment_length,home_ownership,annual_inc,verification_status,issue_date,loan_status,payment_plan,purpose,dti,delinq_2yrs,earliest_credit_line,inq_last_6mths,open_accounts,total_accounts,out_prncp,out_prncp_inv,total_payment,total_payment_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,last_credit_pull_date,collections_12_mths_ex_med,policy_code,application_type
0,38676116,41461848,8000,8000.0,8000.0,36,7.49,248.82,A,A4,5,MORTGAGE,46000.0,Not Verified,2021-01-01,Current,n,credit_card,19.54,2.0,1987-10-01,0,12,27,5529.70,5529.70,2982.51,2982.51,2470.30,512.21,0,0.0,0.0,2022-01-01,248.82,2022-01-01,0,1,INDIVIDUAL
1,38656203,41440010,13200,13200.0,13200.0,36,6.99,407.52,A,A3,9,RENT,50000.0,Not Verified,2021-01-01,Current,n,credit_card,24.20,0.0,2001-09-01,0,15,31,9102.83,9102.83,4885.11,4885.11,4097.17,787.94,0,0.0,0.0,2022-01-01,407.52,2022-01-01,0,1,INDIVIDUAL
2,38656154,41439961,16000,16000.0,16000.0,36,7.49,497.63,A,A4,8,MORTGAGE,73913.0,Source Verified,2021-01-01,Fully Paid,n,credit_card,16.92,0.0,1998-09-01,0,7,18,0.00,0.00,16824.54,16824.54,16000.00,824.54,0,0.0,0.0,2021-10-01,12850.16,2021-10-01,0,1,INDIVIDUAL
3,38656128,41439934,15000,15000.0,15000.0,36,14.31,514.93,C,C4,1,RENT,42000.0,Source Verified,2021-01-01,Fully Paid,n,debt_consolidation,35.52,0.0,2008-06-01,0,6,13,0.00,0.00,15947.47,15947.47,15000.00,947.47,0,0.0,0.0,2021-06-01,13899.67,2021-06-01,0,1,INDIVIDUAL
4,38656121,41439927,15000,15000.0,15000.0,36,6.03,456.54,A,A1,10,MORTGAGE,145000.0,Verified,2021-01-01,Current,n,debt_consolidation,3.33,0.0,2002-04-01,0,23,50,10297.47,10297.47,5473.46,5473.46,4702.53,770.93,0,0.0,0.0,2022-01-01,456.54,2022-01-01,0,1,INDIVIDUAL


In [49]:
# calculate basic loan summary information
percentage_recouped_inv = round(len(loans_data[loans_data['out_prncp_inv']==0])/len(loans_data)*100,2)
total_funded = loans_data['funded_amount'].sum()
funded_percent_inv = round((loans_data['out_prncp_inv'].sum() / loans_data['funded_amount_inv'].sum())*100,2)

print(f'The percentage of loans recovered against the investor funding is {percentage_recouped_inv}%')
print(f'The percentage of investor funding returned is {funded_percent_inv}%')
print(f'The total amount funded is {total_funded:,}')
#calculate loan value in 6 months for each loan

loans_data['remaining_in_6mths'] = ''
for index, row in loans_data.iterrows():
    value = row['out_prncp']
    mths = 6
    while mths > 0:
        value = (value * (1+(row['int_rate']/12/100))) - row['instalment']
        mths -= 1
    
    loans_data.at[index, 'remaining_in_6mths'] = value

loans_data['remaining_in_6mths'] = np.where(loans_data['remaining_in_6mths']<=0,0,loans_data['remaining_in_6mths'])

funds_returned_6mths = round(((loans_data['remaining_in_6mths'].sum() + loans_data['out_prncp_inv'].sum()) / loans_data['funded_amount_inv'].sum())*100,2)

loans_data['status_6mths'] = np.where(loans_data['remaining_in_6mths']>0,"Open","Recovered")    




pie_fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

pie_fig.add_trace(go.Pie(labels=['Returned','Open'], values=[funded_percent_inv,100-funded_percent_inv], name="Current Loans Recovered vs Investor funding"),
              row=1, col=1)
pie_fig.add_trace(go.Pie(labels=['Returned','Open'], values=[funds_returned_6mths,100-funds_returned_6mths], name="Projection of Funds Recovered in 6 months vs Investor funding"),
              row=1, col=2)

# Use `hole` to create a donut-like pie chart
pie_fig.update_traces(hole=.4, hoverinfo="label+percent+name")

pie_fig.update_layout(
    title_text="<b>Investor Funds Returned:</b> Current vs. 6 month projection",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Current', x=0.18, y=0.5, font_size=18, showarrow=False),
                 dict(text='Projection', x=0.83, y=0.5, font_size=18, showarrow=False)])

pie_fig.show(config={'displayModeBar':False})




The percentage of loans recovered against the investor funding is 61.75%
The percentage of investor funding returned is 19.61%
The total amount funded is 398,914,525.0


In [30]:
display(loans_data.head())

,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,employment_length,home_ownership,annual_inc,verification_status,issue_date,loan_status,payment_plan,purpose,dti,delinq_2yrs,earliest_credit_line,inq_last_6mths,open_accounts,total_accounts,out_prncp,out_prncp_inv,total_payment,total_payment_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,last_credit_pull_date,collections_12_mths_ex_med,policy_code,application_type,remaining_in_6mths,status_6mths
0,38676116,41461848,8000,8000.0,8000.0,36,7.49,248.82,A,A4,5,MORTGAGE,46000.0,Not Verified,2021-01-01,Current,n,credit_card,19.54,2.0,1987-10-01,0,12,27,5529.70,5529.70,2982.51,2982.51,2470.30,512.21,0,0.0,0.0,2022-01-01,248.82,2022-01-01,0,1,INDIVIDUAL,4223.635153,Open
1,38656203,41440010,13200,13200.0,13200.0,36,6.99,407.52,A,A3,9,RENT,50000.0,Not Verified,2021-01-01,Current,n,credit_card,24.20,0.0,2001-09-01,0,15,31,9102.83,9102.83,4885.11,4885.11,4097.17,787.94,0,0.0,0.0,2022-01-01,407.52,2022-01-01,0,1,INDIVIDUAL,6944.6382,Open
2,38656154,41439961,16000,16000.0,16000.0,36,7.49,497.63,A,A4,8,MORTGAGE,73913.0,Source Verified,2021-01-01,Fully Paid,n,credit_card,16.92,0.0,1998-09-01,0,7,18,0.00,0.00,16824.54,16824.54,16000.00,824.54,0,0.0,0.0,2021-10-01,12850.16,2021-10-01,0,1,INDIVIDUAL,0,Recovered
3,38656128,41439934,15000,15000.0,15000.0,36,14.31,514.93,C,C4,1,RENT,42000.0,Source Verified,2021-01-01,Fully Paid,n,debt_consolidation,35.52,0.0,2008-06-01,0,6,13,0.00,0.00,15947.47,15947.47,15000.00,947.47,0,0.0,0.0,2021-06-01,13899.67,2021-06-01,0,1,INDIVIDUAL,0,Recovered
4,38656121,41439927,15000,15000.0,15000.0,36,6.03,456.54,A,A1,10,MORTGAGE,145000.0,Verified,2021-01-01,Current,n,debt_consolidation,3.33,0.0,2002-04-01,0,23,50,10297.47,10297.47,5473.46,5473.46,4702.53,770.93,0,0.0,0.0,2022-01-01,456.54,2022-01-01,0,1,INDIVIDUAL,7837.982083,Open
